In [1]:
#Import the GST module -- you probably want this at the beginning of every notebook
import GST

In [2]:
#There three basic ways to create DataSet objects in GST.
# 1) By creating an empty DataSet and manually adding counts corresponding to gate strings
# 2) By loading from a text-format dataset file
# 3) By using a GateSet to generate "fake" data 

# We'll do each of these in turn

In [3]:
#1) Creating from scratch
ds1 = GST.DataSet(spamLabels=['plus','minus'])
ds1.addCountDict( ('Gx',), {'plus': 10, 'minus': 90} )
ds1.addCountDict( ('Gx','Gy'), {'plus': 40, 'minus': 60} )
ds1.doneAddingData()

In [4]:
#2) By creating and loading a text-format dataset file
dataset_txt = \
"""## Columns = plus count, count total
{} 0 100
Gx 10 90
GxGy 40 60
Gx^4 20 90
"""
open("tutorial_files/Example_TinyDataset.txt","w").write(dataset_txt)
ds2 = GST.loadDataset("tutorial_files/Example_TinyDataset.txt")

Loading tutorial_files/Example_TinyDataset.txt: 100%


In [5]:
#3) By generating fake data (using our example gate list string from the previous tutorial)
gateset = GST.loadGateset("tutorial_files/Example_Gateset.txt")
depol_gateset = GST.GateSetTools.depolarizeGateset(gateset, noise=0.1)
gateStringList = GST.loadGatestringList("tutorial_files/Example_GatestringList.txt")
ds3 = GST.generateFakeData(depol_gateset, gateStringList, nSamples=1000, sampleError='binomial', seed=100)
ds3b = GST.generateFakeData(depol_gateset, gateStringList, nSamples=50, sampleError='binomial', seed=100)

#write this dataset to a file for later tutorials
GST.writeDatasetFile("tutorial_files/Example_Dataset.txt", gateStringList, ds3, spamLabelOrder=['plus','minus']) 
GST.writeDatasetFile("tutorial_files/Example_Dataset_LowCnts.txt", gateStringList, ds3b) 

In [6]:
print "Dataset1:\n",ds1
print "Dataset2:\n",ds2
print "Dataset3 is too big to print, so here it is truncated to Dataset2's strings\n", ds3.truncate(ds2.keys())

Dataset1:
('Gx',)  :  {'plus': 10.0, 'minus': 90.0}
('Gx', 'Gy')  :  {'plus': 40.0, 'minus': 60.0}


Dataset2:
()  :  {'plus': 0.0, 'minus': 100.0}
('Gx',)  :  {'plus': 10.0, 'minus': 80.0}
('Gx', 'Gy')  :  {'plus': 40.0, 'minus': 20.0}
('Gx', 'Gx', 'Gx', 'Gx')  :  {'plus': 20.0, 'minus': 70.0}


Dataset3 is too big to print, so here it is truncated to Dataset2's strings
()  :  {'plus': 0.0, 'minus': 1000.0}
('Gx',)  :  {'plus': 499.0, 'minus': 501.0}
('Gx', 'Gy')  :  {'plus': 504.0, 'minus': 496.0}
('Gx', 'Gx', 'Gx', 'Gx')  :  {'plus': 171.0, 'minus': 829.0}


